In [1]:
import requests
import pandas as pd
import urllib
import json
from dateutil.parser import parse

###### looking at the addCrashLayers.js file that is listed in the page source script of http://app01.cityofboston.gov/VisionZero/ the data is coming from an esri gis rest api as indicated by this url http://app01.cityofboston.gov/VisionZero/js/addCrashLayers.js 

In [2]:
#method to extract a batch of 999 instances because there is a 1000 instance limit on the esri rest api queries
def getFeatures(fidmax):
    
    fid_max = fidmax
    fid_min = fid_max-1000
    
    url = 'http://gpd01.cityofboston.gov:6080/arcgis/rest/services/all_crashes_analysis/MapServer/6/query?where='+ \
        str(fid_min)+'%3C+FID+AND+FID+%3C'+str(fid_max)+'&text=&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope& \
        inSR=&spatialRel=esriSpatialRelIntersects&relationParam=&outFields=*&returnGeometry=true&maxAllowableOffset=& \
        geometryPrecision=&outSR=&returnIdsOnly=false&returnCountOnly=false&orderByFields=&groupByFieldsForStatistics=& \
        outStatistics=&returnZ=false&returnM=false&gdbVersion=&returnDistinctValues=false&f=json'
        
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    
    return data

In [3]:
# setting up lists for relevant data to extract
Count = []
Date = []
FID = []
Incident = []
Mode = []
Lat = []
Lng = []
Crds = []
Day = []
Time = []

#set initial value of 1000 to extract the first 999 query results
fidmax = 1000
data = getFeatures(fidmax) 

# continue to loop through the query result batches until it stops providing data
while (len(data['features'])) > 0:

    fidmax+=1000 # increase to the next 999 query results
    data = getFeatures(fidmax)

    for i in range(len(data['features'])):

        # parse the date into datetime format
        d = parse(data['features'][i]['attributes']['Date'])
        year = d.strftime("%Y")
        
        # only extract data if the year is 2016
        if year == '2016':
            Date.append(data['features'][i]['attributes']['Date'])
            Day.append(d.strftime("%Y-%m-%d"))
            Time.append(d.strftime('%H:%M:%S %p'))

            Count.append(data['features'][i]['attributes']['Count'])
            FID.append(data['features'][i]['attributes']['FID'])
            Incident.append(data['features'][i]['attributes']['Incident'])
            Mode.append(data['features'][i]['attributes']['Mode'])

            Lat.append(data['features'][i]['geometry']['x'])
            Lng.append(data['features'][i]['geometry']['y'])
            Crds.append((data['features'][i]['geometry']['x'],
                          data['features'][i]['geometry']['y']))

In [4]:
# convert compiled raw data into an initial dataframe
df_raw = pd.DataFrame(list(map(list, zip(Incident, Date, Mode, Count, Crds))))
df_raw.columns = ['Incident', 'Datetime', 'Mode', 'Count', 'Crds']
df_raw.head()

,Incident,Datetime,Mode,Count,Crds
0,160010115,1/1/2016 2:36:37 AM,mv,0,"(-7910572.43766, 5208732.15296)"
1,160010146,1/1/2016 4:07:00 AM,mv,0,"(-7912769.09751, 5203023.23632)"
2,160010147,1/1/2016 4:13:50 AM,mv,0,"(-7911712.67505, 5210924.29623)"
3,160010167,1/1/2016 5:09:03 AM,mv,0,"(-7911502.36108, 5207733.60732)"
4,160010180,1/1/2016 6:02:48 AM,mv,0,"(-7916797.09353, 5204886.86142)"


In [5]:
# convert compiled data into revised format with additional null columns
df_final =pd.DataFrame(list(map(list, zip(Incident, Day, Time, Mode, Count, Lat, Lng))))
df_final.columns = ['INCIDENT', 'DATE', 'TIME', 'MODE', 'COUNT', 'LATITUDE', 'LONGITUDE']
df_final['REPORTID'] = None
df_final['USERID'] = None
df_final['ADDEDBY'] = None
df_final['HOSTID'] = None
df_final['RADIUSIMPACT'] = None
df_final['DESCRIPTION'] = None
df_final['PRICE'] = None
df_final['REPORTSTATUS'] = None
df_final['CREATEDTIME'] = None
df_final['ADDRESS'] = None
df_final['TOTALNUMBERINJURED'] = None
df_final['TOTALNUMBERKILLED'] = None
df_final['VEHICLETYPECODE'] = None
df_final.head()

,INCIDENT,DATE,TIME,MODE,COUNT,LATITUDE,LONGITUDE,REPORTID,USERID,ADDEDBY,HOSTID,RADIUSIMPACT,DESCRIPTION,PRICE,REPORTSTATUS,CREATEDTIME,ADDRESS,TOTALNUMBERINJURED,TOTALNUMBERKILLED,VEHICLETYPECODE
0,160010115,2016-01-01,02:36:37 AM,mv,0,-7.910572e+06,5.208732e+06,None,None,None,None,None,None,None,None,None,None,None,None,None
1,160010146,2016-01-01,04:07:00 AM,mv,0,-7.912769e+06,5.203023e+06,None,None,None,None,None,None,None,None,None,None,None,None,None
2,160010147,2016-01-01,04:13:50 AM,mv,0,-7.911713e+06,5.210924e+06,None,None,None,None,None,None,None,None,None,None,None,None,None
3,160010167,2016-01-01,05:09:03 AM,mv,0,-7.911502e+06,5.207734e+06,None,None,None,None,None,None,None,None,None,None,None,None,None
4,160010180,2016-01-01,06:02:48 AM,mv,0,-7.916797e+06,5.204887e+06,None,None,None,None,None,None,None,None,None,None,None,None,None


In [7]:
# convert the final dataframe to an excel file
df_final.to_excel('boston_collision_data_2016.xlsx',index=False)